## **Customer Churn Prediction System using Deep Learning**

In [ ]:


# Importing the dependecies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import tensorflow as tf

In [ ]:
df = pd.read_csv(r"../datasets/Churn_Modelling.csv")
df.head()

### **Data Preprocessing**

In [ ]:
df.info()
df.drop_duplicates()

In [ ]:
df= df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
df

In [ ]:
df = pd.get_dummies(df, columns=['Geography'], drop_first = True)
df

In [ ]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Geography_Germany'] = le.fit_transform(df['Geography_Germany'])
df['Geography_Spain'] = le.fit_transform(df['Geography_Spain'])
df        

In [ ]:
col = 'Exited'
df = df[[i for i in df.columns if i!=col]+ [col]]
df

In [ ]:
# (0,0) => France,     (0,1) => Spain,     (1,0) = Germany,         0 = Female,   1 => Male

x = df.iloc[:, :-1]
y = df['Exited']
df['Exited'].value_counts()

In [ ]:
sm = SMOTE()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42, stratify = y)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)


In [ ]:
sc = StandardScaler()
scaling = ['CreditScore', 'Balance', 'EstimatedSalary']
x_train_res[scaling] = sc.fit_transform(x_train_res[scaling])
x_test[scaling] = sc.transform(x_test[scaling])


model = Sequential()

model.add(Dense(64, activation='relu', input_dim = 11))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)


In [ ]:
history = model.fit(
    x_train_res,
    y_train_res,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


In [ ]:
y_pred = (model.predict(x_test) > 0.5).astype(int)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

print("\nROC-AUC Score:", roc_auc_score(y_test, y_pred))


In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history['accuracy'])

In [ ]:
plt.plot(history.history['loss'])